# TTN遺伝子の両端の配列に着目しmotif配列の検出を行う

### 06_intron_edge_motif_pca.ipynbの内容
各イントロン両端の特定の領域に存在する特徴的な配列を検出するために主成分分析を行う

## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count
from lib.geneinfo import gene_id

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
gene_id('TTN')

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
# gbk.read_gbk('../data/gbk/human_ttn.gb')
gbk.get_gbk('7273')

In [ ]:
# 各バリアントのエクソン数を取得
gbk.variant_exons()

## すべてのイントロンの両端n塩基を抽出し、それらをもとに主成分分析を行う

In [ ]:
# 各イントロンの両端n塩基を抽出し
n = 30

# 各イントロンの5-prime側のn塩基を抽出 
left_edges = []
for i in range(gbk.intron_num()):
    left_edges.append(str(gbk.intron_seq(i+1)[:n]))
    
# 各イントロンの3-prime側のn塩基を抽出
right_edges = []
for i in range(gbk.intron_num()):
    right_edges.append(str(gbk.intron_seq(i+1)[-n:]))

## 5'側のn塩基の配列をもとに主成分分析

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences = np.array([one_hot_encode(seq) for seq in left_edges])

# 主成分分析（PCA）
pca_l = PCA(n_components=2)  # 2つの主成分を取得
principal_components = pca_l.fit_transform(encoded_sequences)

# 主成分をデータフレームに変換
df_pca_l = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 結果をプロット
fig = px.scatter(df_pca_l, x='PC1', y='PC2', hover_name=[f"intron_{i+1}" for i in range(gbk.intron_num())] )
fig.show()


**主成分１の可視化**

In [ ]:
# 各イントロンのPC1の値を棒グラフにする
fig = px.bar(x=df_pca_l.index, y=df_pca_l['PC1'])
fig.show()

In [ ]:
# 各バリアントのエクソンを表示
gbk.transcript_variants()

**主成分1の因子負荷量の可視化**

In [ ]:
# 因子負荷量の計算
loadings_l = pca_l.components_.T * np.sqrt(pca_l.explained_variance_)

# 因子負荷量を縦軸に棒グラフを作成
loadings_ldf = pd.DataFrame(loadings_l, columns=['PC1', 'PC2'])
ldf_l = pd.DataFrame(loadings_ldf)
ldf_l.sort_values('PC1')
fig = px.bar(x=ldf_l.index, y=ldf_l['PC1'])
fig.show()

**主成分1の因子負荷量に基づいてsequence　logoを作成**

In [ ]:
# ワンホットエンコーディングをデコードする
# 負荷量の最も高い塩基を取得し、配列を作成する
b_df_l = pd.DataFrame(ldf_l['PC1'].to_numpy().reshape(n,4))
b_df_l.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(b_df_l, shade_below=.5, fade_below=.5)

## 3'側のn塩基の配列をもとに主成分分析

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences = np.array([one_hot_encode(seq) for seq in right_edges])

# 主成分分析（PCA）
pca_r = PCA(n_components=2)  # 2つの主成分を取得
principal_components = pca_r.fit_transform(encoded_sequences)

# 主成分をデータフレームに変換
df_pca_r = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# 結果をプロット
fig = px.scatter(df_pca_r, x='PC1', y='PC2', hover_name=[f"intron_{i+1}" for i in range(gbk.intron_num())] )
fig.show()

**主成分１の可視化**

In [ ]:
# 各イントロンのPC1の値を棒グラフにする
fig = px.bar(x=df_pca_r.index, y=df_pca_r['PC1'])
fig.show()

In [ ]:
# 各バリアントのエクソンを表示
gbk.transcript_variants()

**主成分1の因子負荷量の可視化**

In [ ]:
# 因子負荷量の計算
loadings_r = pca_r.components_.T * np.sqrt(pca_r.explained_variance_)

# 因子負荷量を縦軸に棒グラフを作成
loadings_rdf = pd.DataFrame(loadings_r, columns=['PC1', 'PC2'])
ldf_r = pd.DataFrame(loadings_rdf)
ldf_r.sort_values('PC1')
fig = px.bar(x=ldf_r.index, y=ldf_r['PC1'])
fig.show()

**主成分1の因子負荷量に基づいてsequence　logoを作成**

In [ ]:
# ワンホットエンコーディングをデコードする
# 負荷量の最も高い塩基を取得し、配列を作成する
b_df_r = pd.DataFrame(ldf_r['PC1'].to_numpy().reshape(n,4))
b_df_r.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(b_df_r, shade_below=.5, fade_below=.5)

## 上記の結果をもとにした検証

In [ ]:
# 上記の配列の一部が末尾n塩基に含まれているイントロンを抽出
mtf = "ACTAATATCTT"
intron_mtf = []
for i in range(gbk.intron_num()):
    if mtf in gbk.intron_seq(i+1)[-n:]:
        intron_mtf.append(i+1)

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)
mtf = "ACTAATATCTT"
intron_mtf = []
for i in range(gbk.intron_num()):
    if mtf in gbk.intron_seq(i+1)[-30:]:
        intron_mtf.append(1)
    else:
        intron_mtf.append(0)
        
x = [i+1 for i in range(len(intron_mtf))]
y = intron_mtf
df = pd.DataFrame({"introns": x, "existence":y})
# ホバーテンプレートの設定
hovertemplate = 'Intron: %{x}<extra></extra>'
fig.add_trace(go.Bar(x=df['introns'], y=df['existence'],  hovertemplate=hovertemplate))
fig.update_layout(width=800, height=250, title_text="Introns with sequence ACTAATATCTT  at the 3' edge(NM_001267550.2)" )
fig.show()

In [ ]:
# 上記の配列の一部を含むイントロンの部位を可視化
gbk.set_interest_seq('ACTAATATCTT')
gbk.heatmap()

## Proporton Spliced In
参考論文 [Contribution of Noncanonical Splice Variants to TTN Truncating Variant Cardiomyopathy](https://www.ahajournals.org/doi/full/10.1161/CIRCGEN.121.003389)<br>
<br>
**Figure 5A:**<br>
[GTEX](https://gtexportal.org/home/)データベース情報をもとに骨格筋、左心室心筋、右心房心筋のmRNAのスプライスインされたエクソンの割合を示すグラフ

![PSI](../data/pic/Figure5A.png)

In [ ]:
def match(seq1, seq2):
    match = 0
    for i,j in zip(seq1, seq2):
        if i == j:
            match += 1
    return match/len(seq1)


In [ ]:
seq = gbk.gDNA_seq()

In [ ]:

# mtf = "ACTAATATCTTTCAAG"
mtf = "AAAAATACTAATATCTTTCAAG"
# mtf = "ATCTTTCAAG"

result = []
for i in gbk.intron_list():
    tmp_seq = seq[i[0]:i[1]][-len(mtf):]
    result.append(match(mtf, tmp_seq))
px.bar(x=[i+1 for i in range(len(result))], y=result)